<a href="https://colab.research.google.com/github/Ashar-riaz/Streaming-Avatar/blob/main/UI_Animation_according_to_wavelength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install aspose-words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00


In [ ]:
Atext=input("Enter your text: ")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

prompt = Atext
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [ ]:
print(response)

In [ ]:
from transformers import pipeline
import torch
import soundfile as sf

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

# Replace this with your custom speaker embedding
custom_speaker_embedding = torch.randn(1, 512)  # Example of a random speaker embedding
max_length = 512  # You might need to adjust this value
truncated_response = response[:max_length]

speech = synthesiser(truncated_response, forward_params={"speaker_embeddings": custom_speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

In [ ]:
import IPython.display as ipd

ipd.Audio("speech.wav")

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Function to check if an image is loaded correctly
def check_image_loaded(image, image_path):
    if image is None:
        raise FileNotFoundError(f"Image at path {image_path} could not be loaded.")
    return image

# Load images
avatar_image_path = "/content/face.png"
mouth_image_path_1 = "/content/mouth.png"

avatar_image = cv2.imread(avatar_image_path)
mouth_image_1 = cv2.imread(mouth_image_path_1)

avatar_image = check_image_loaded(avatar_image, avatar_image_path)
mouth_image_1 = check_image_loaded(mouth_image_1, mouth_image_path_1)

# Print sizes to verify
print("Avatar image shape:", avatar_image.shape)
print("Mouth image shape:", mouth_image_1.shape)

# Find the smallest dimensions
min_height = min(avatar_image.shape[0], mouth_image_1.shape[0])
min_width = min(avatar_image.shape[1], mouth_image_1.shape[1])

# Resize images to the smallest dimensions
avatar_image_resized = cv2.resize(avatar_image, (min_width, min_height))
mouth_image_resized = cv2.resize(mouth_image_1, (min_width, min_height))

# Save the resized images
cv2.imwrite("avatar_image_reshaped.jpg", avatar_image_resized)
cv2.imwrite("mouth_image_reshaped.jpg", mouth_image_resized)

# Display the resized images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(avatar_image_resized, cv2.COLOR_BGR2RGB))
plt.title("Resized Avatar Image")
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(mouth_image_resized, cv2.COLOR_BGR2RGB))
plt.title("Resized Mouth Image")
plt.show()


In [ ]:
import cv2
import numpy as np
import librosa
import moviepy.editor as mp
from IPython.display import display, Video

# Function to check if an image is loaded correctly
def check_image_loaded(image, image_path):
    if image is None:
        raise FileNotFoundError(f"Image at path {image_path} could not be loaded.")
    return image

# Load the images
avatar_image_path = "/content/avatar_image_reshaped.jpg"
mouth_open_path = "/content/mouth_image_reshaped.jpg"

avatar_image = cv2.imread(cv2.samples.findFile(avatar_image_path))
mouth_open_image = cv2.imread(cv2.samples.findFile(mouth_open_path))

avatar_image = check_image_loaded(avatar_image, avatar_image_path)
mouth_open_image = check_image_loaded(mouth_open_image, mouth_open_path)

# Ensure both images are the same size
target_size = min(avatar_image.shape[:2], mouth_open_image.shape[:2])
avatar_image = cv2.resize(avatar_image, (target_size[1], target_size[0]))
mouth_open_image = cv2.resize(mouth_open_image, (target_size[1], target_size[0]))

# Load the audio file
audio_file = "/content/speech.wav"
audio, sample_rate = librosa.load(audio_file)

# Extract the amplitude envelope
amplitude_envelope = np.abs(librosa.core.amplitude_to_db(audio))

# Create a video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("lipsync.mp4", fourcc, 30, (avatar_image.shape[1], avatar_image.shape[0]))

# Iterate over the audio frames and update the avatar's mouth shape
frame_duration = 1.0 / 30  # Duration of each video frame
total_frames = int(len(audio) / (sample_rate * frame_duration))

for i in range(total_frames):
    start_sample = int(i * sample_rate * frame_duration)
    end_sample = int((i + 1) * sample_rate * frame_duration)
    frame_audio = audio[start_sample:end_sample]

    # Determine the mouth shape based on the amplitude
    if np.mean(np.abs(frame_audio)) > 0.025:  # Adjust the threshold as needed
        frame = mouth_open_image
    else:
        frame = avatar_image

    # Write the frame to the video
    video_writer.write(frame)

# Release the video writer
video_writer.release()

# Add audio to the video
video = mp.VideoFileClip("lipsync.mp4")
audio = mp.AudioFileClip(audio_file)
final_video = video.set_audio(audio)
final_video.write_videofile("lipsync_with_audio.mp4", codec='libx264')


In [ ]:
from IPython.display import display, Video

# Assuming the video file is named 'lipsync_with_audio.mp4'
video_path = 'lipsync_with_audio.mp4'  # Replace with the actual name of your uploaded file
display(Video(video_path, embed=True, width=600, height=400))

In [ ]:
!pip install gtts

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from gtts import gTTS
import os
import cv2
import numpy as np
import librosa
import moviepy.editor as mp
from IPython.display import display, Video
def avtar(audio_path):
    # Function to check if an image is loaded correctly
    def check_image_loaded(image, image_path):
        if image is None:
            raise FileNotFoundError(f"Image at path {image_path} could not be loaded.")
        return image

    # Load the images
    avatar_image_path = "avatar_image_reshaped.jpg"
    mouth_open_path = "mouth_image_reshaped.jpg"

    avatar_image = cv2.imread(cv2.samples.findFile(avatar_image_path))
    mouth_open_image = cv2.imread(cv2.samples.findFile(mouth_open_path))

    avatar_image = check_image_loaded(avatar_image, avatar_image_path)
    mouth_open_image = check_image_loaded(mouth_open_image, mouth_open_path)

    # Ensure both images are the same size
    target_size = min(avatar_image.shape[:2], mouth_open_image.shape[:2])
    avatar_image = cv2.resize(avatar_image, (target_size[1], target_size[0]))
    mouth_open_image = cv2.resize(mouth_open_image, (target_size[1], target_size[0]))

    # Load the audio file
    audio, sample_rate = librosa.load(audio_path)

    # Extract the amplitude envelope
    amplitude_envelope = np.abs(librosa.core.amplitude_to_db(audio))

    # Create a video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter("lipsync.mp4", fourcc, 30, (avatar_image.shape[1], avatar_image.shape[0]))

    # Iterate over the audio frames and update the avatar's mouth shape
    frame_duration = 1.0 / 30  # Duration of each video frame
    total_frames = int(len(audio) / (sample_rate * frame_duration))

    for i in range(total_frames):
        start_sample = int(i * sample_rate * frame_duration)
        end_sample = int((i + 1) * sample_rate * frame_duration)
        frame_audio = audio[start_sample:end_sample]

        # Determine the mouth shape based on the amplitude
        if np.mean(np.abs(frame_audio)) > 0.025:  # Adjust the threshold as needed
            frame = mouth_open_image
        else:
            frame = avatar_image

        # Write the frame to the video
        video_writer.write(frame)

    # Release the video writer
    video_writer.release()

    # Add audio to the video
    video = mp.VideoFileClip("lipsync.mp4")
    audio = mp.AudioFileClip(audio_path)
    final_video = video.set_audio(audio)
    final_video.write_videofile("lipsync_with_audio.mp4", codec='libx264')

    return "lipsync_with_audio.mp4"
def text_to_speech(text):
    # Generate speech from text
    tts = gTTS(text)
    audio_path = "response.mp3"
    tts.save(audio_path)
    return audio_path

def generate_response(input_text):
    prompt = input_text
    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    audio_path = text_to_speech(response)

        # Generate the avatar video
    video_path = avtar(audio_path)

    return response,audio_path,video_path



# Create Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs=["text","audio","video"],
    title="Text to Avatar Speech",
    description="Enter text and get a spoken response by the avatar."
)

iface.launch()